In [1]:
import pandas as pd

data = pd.read_csv("/home/fm-pc-lt-228/Desktop/covid_nepali_tweets/covid19_tweeter_dataset (copy).csv")

In [2]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# import snowballstemmer
import numpy
import os 
import re
import json
import pickle 

model = AutoModelForMaskedLM.from_pretrained("Rajan/NepaliBERT", output_hidden_states = True, return_dict = True, output_attentions = True)

tokenizers = AutoTokenizer.from_pretrained("Rajan/NepaliBERT")

In [3]:
def get_bert_embedding_sentence(input_sentence):
    md = model
    tokenizer = tokenizers
#     md = local_model
#     tokenizer = local_tokenizers
    marked_text = " [CLS] " + input_sentence + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = md(tokens_tensors, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 

        hidden_states = outputs.hidden_states
#         print(hidden_states[-2])
        # second_hidden_states = outputs[2]
    # hidden_states has shape [13 x 1 x 22 x 768]

    # token_vecs is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]
    # get last four layers
#     last_four_layers = [hidden_states[i] for i in (-1,-2, -3,-4)]
    # cast layers to a tuple and concatenate over the last dimension
#     cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.shape)
#     import pdb;pdb.set_trace()
    token_vecs = hidden_states[-2][0]
    # take the mean of the concatenated vector over the token dimension
#     sentence_embedding = torch.mean(cat_hidden_states, dim=0).squeeze()

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
#     sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding.numpy()

In [4]:
data['word_embeddings'] = data['Tweet'].apply(get_bert_embedding_sentence)

In [5]:
data.to_csv("embeddings_data_nepali_BERT.csv")

In [6]:
X,y = data['word_embeddings'], data['Label']

In [7]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

In [8]:
svc = SVC(probability=True)

In [9]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC(probability=True)

In [10]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [11]:
print(confusion_matrix(test_y, svc_pred))

[[2000   44  777]
 [ 298  426  575]
 [ 466   40 2454]]


In [12]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

          -1       0.72      0.71      0.72      2821
           0       0.84      0.33      0.47      1299
           1       0.64      0.83      0.73      2960

    accuracy                           0.69      7080
   macro avg       0.73      0.62      0.64      7080
weighted avg       0.71      0.69      0.68      7080



In [13]:
accuracy_score(test_y, svc_pred)

0.6892655367231638

In [14]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=2, random_state=0)
random_forest.fit(train_X.tolist(), train_y)



RandomForestClassifier(max_depth=2, random_state=0)

In [16]:
random_pred = random_forest.predict(test_X.tolist())
print(accuracy_score(test_y, svc_pred))

print(confusion_matrix(test_y, random_pred))

print(classification_report(test_y, random_pred))

0.6892655367231638
[[1418    0 1403]
 [ 286    9 1004]
 [ 398    0 2562]]
              precision    recall  f1-score   support

          -1       0.67      0.50      0.58      2821
           0       1.00      0.01      0.01      1299
           1       0.52      0.87      0.65      2960

    accuracy                           0.56      7080
   macro avg       0.73      0.46      0.41      7080
weighted avg       0.67      0.56      0.50      7080



# Naive Bayes

In [17]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_X.tolist(), train_y)


GaussianNB()

In [18]:
gnb_pred = gnb.predict(test_X.tolist())
print(accuracy_score(test_y, gnb_pred))

print(confusion_matrix(test_y, gnb_pred))

print(classification_report(test_y, gnb_pred))

0.5553672316384181
[[1355  471  995]
 [ 192  587  520]
 [ 364  606 1990]]
              precision    recall  f1-score   support

          -1       0.71      0.48      0.57      2821
           0       0.35      0.45      0.40      1299
           1       0.57      0.67      0.62      2960

    accuracy                           0.56      7080
   macro avg       0.54      0.53      0.53      7080
weighted avg       0.58      0.56      0.56      7080



# Ada Boost

In [19]:
from sklearn.ensemble import AdaBoostClassifier
ada_boost_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
ada_boost_clf.fit(train_X.tolist(), train_y)


AdaBoostClassifier(n_estimators=100, random_state=0)

In [20]:
adaboost_pred = ada_boost_clf.predict(test_X.tolist())
print(accuracy_score(test_y, adaboost_pred))

print(confusion_matrix(test_y, adaboost_pred))

print(classification_report(test_y, adaboost_pred))

0.6211864406779661
[[1812  146  863]
 [ 358  420  521]
 [ 623  171 2166]]
              precision    recall  f1-score   support

          -1       0.65      0.64      0.65      2821
           0       0.57      0.32      0.41      1299
           1       0.61      0.73      0.67      2960

    accuracy                           0.62      7080
   macro avg       0.61      0.57      0.57      7080
weighted avg       0.62      0.62      0.61      7080



# Decision Tree

In [21]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree.fit(train_X.tolist(), train_y)


DecisionTreeClassifier(max_depth=2, random_state=0)

In [22]:
decision_pred = decision_tree.predict(test_X.tolist())
print(accuracy_score(test_y, decision_pred))

print(confusion_matrix(test_y, decision_pred))

print(classification_report(test_y, decision_pred))

0.5214689265536723
[[2021    0  800]
 [ 792    0  507]
 [1289    0 1671]]
              precision    recall  f1-score   support

          -1       0.49      0.72      0.58      2821
           0       0.00      0.00      0.00      1299
           1       0.56      0.56      0.56      2960

    accuracy                           0.52      7080
   macro avg       0.35      0.43      0.38      7080
weighted avg       0.43      0.52      0.47      7080



/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

# Extra Trees Classifier

In [23]:
from sklearn.ensemble import ExtraTreesClassifier
etc_clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
etc_clf.fit(train_X.tolist(), train_y)


ExtraTreesClassifier(random_state=0)

In [24]:
etc_pred = etc_clf.predict(test_X.tolist())
print(accuracy_score(test_y, etc_pred))

print(confusion_matrix(test_y, etc_pred))

print(classification_report(test_y, etc_pred))

0.6529661016949152
[[1796   40  985]
 [ 302  374  623]
 [ 462   45 2453]]
              precision    recall  f1-score   support

          -1       0.70      0.64      0.67      2821
           0       0.81      0.29      0.43      1299
           1       0.60      0.83      0.70      2960

    accuracy                           0.65      7080
   macro avg       0.71      0.58      0.60      7080
weighted avg       0.68      0.65      0.64      7080



# Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(train_X.tolist(), train_y)

/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=100000.0)

In [26]:
lr_pred = logreg.predict(test_X.tolist())
print(accuracy_score(test_y, lr_pred))

print(confusion_matrix(test_y, lr_pred))

print(classification_report(test_y, lr_pred))

0.6785310734463277
[[1998  106  717]
 [ 309  504  486]
 [ 527  131 2302]]
              precision    recall  f1-score   support

          -1       0.71      0.71      0.71      2821
           0       0.68      0.39      0.49      1299
           1       0.66      0.78      0.71      2960

    accuracy                           0.68      7080
   macro avg       0.68      0.62      0.64      7080
weighted avg       0.68      0.68      0.67      7080



# KNN

In [27]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X.tolist(), train_y)

KNeighborsClassifier(n_neighbors=3)

In [28]:
neigh_pred = neigh.predict(test_X.tolist())
print(accuracy_score(test_y, neigh_pred))

print(confusion_matrix(test_y, neigh_pred))

print(classification_report(test_y, neigh_pred))

0.6446327683615819
[[2047  147  627]
 [ 421  460  418]
 [ 745  158 2057]]
              precision    recall  f1-score   support

          -1       0.64      0.73      0.68      2821
           0       0.60      0.35      0.45      1299
           1       0.66      0.69      0.68      2960

    accuracy                           0.64      7080
   macro avg       0.63      0.59      0.60      7080
weighted avg       0.64      0.64      0.64      7080



# MLP

In [29]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=300)
mlp.fit(train_X.tolist(), train_y)

MLPClassifier(max_iter=300, random_state=1)

In [30]:
mlp_pred = mlp.predict(test_X.tolist())
print(accuracy_score(test_y, mlp_pred))

print(confusion_matrix(test_y, mlp_pred))

print(classification_report(test_y, mlp_pred))

0.6601694915254237
[[1939  269  613]
 [ 258  659  382]
 [ 576  308 2076]]
              precision    recall  f1-score   support

          -1       0.70      0.69      0.69      2821
           0       0.53      0.51      0.52      1299
           1       0.68      0.70      0.69      2960

    accuracy                           0.66      7080
   macro avg       0.64      0.63      0.63      7080
weighted avg       0.66      0.66      0.66      7080

